In [1]:
%matplotlib inline

from misc import *

from ipfn import *

from itertools import product

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#Load the survey data

index1 = preprocess(pd.read_excel('data/RawData/IN10001.xls'))
index4 = preprocess(pd.read_excel('data/RawData/IN10004.xls'))
index5 = preprocess(pd.read_excel('data/RawData/IN10005.xls'))
index6 = preprocess(pd.read_excel('data/RawData/IN10006.xls'))
index7 = preprocess(pd.read_excel('data/RawData/IN10007.xls'))

#Create a new dataframe that only contains the demographic information that's included in each survey

common_columns = list(set(index1.columns).intersection(set(index7.columns)))

common_data = pd.concat([index1[common_columns],
                         index4[common_columns],
                         index5[common_columns],
                         index6[common_columns],
                         index7[common_columns]
                        ])

#Drop the columns that don't contain demographic information

common_data.drop(['ID', 'ADID IDFA', 'Time Started', 'Time Finished'], axis=1, inplace=True)

#Drop the columns where area is missing since the distributions for each demographic doesn't differ much

common_data = common_data.loc[common_data['Area'] != 'Unknown', :]


In [3]:
"""
Raking using the ipfn library requires the following steps demonstrated below:

1. The survey data must be grouped by the variables of interest to obtain the sample marginal frequencies.
2. The resulting marginal frequency column must be named "total" or ipfn will raise an error.
3. Marginal frequencies for each individual variable should be saved as a pandas series.
4. Joint frequencies for pairs of variables should be saved as a pandas series
5. Create a list of the pandas series that contain all of the joint and marginal frequencies.
6. Create a list of lists, with each list containing the dataframe column names corresponding to a joint/marginal frequency.
"""

#Group each survey by gender, race, and age range to obtain the sample marginal frequencies

freqs = pd.DataFrame(common_data.groupby(['Area', 'Gender', 'Race', 'Age', 'Education']).size())
freqs = freqs.reset_index()
freqs.columns = ['Area', 'Gender', 'Race', 'Age', 'Education', 'total']
freqs


,Area,Gender,Race,Age,Education,total
0,Alabama,female,black,25 - 34,no bachelors,3
1,Alabama,female,black,35 - 44,bachelors,1
2,Alabama,female,black,35 - 44,no bachelors,3
3,Alabama,female,black,45 - 54,no bachelors,1
4,Alabama,female,black,> 54,no bachelors,1
5,Alabama,female,hispanic,25 - 34,no bachelors,1
6,Alabama,female,other,18 - 24,bachelors,1
7,Alabama,female,white,18 - 24,no bachelors,5
8,Alabama,female,white,25 - 34,no bachelors,4
9,Alabama,female,white,35 - 44,no bachelors,5


In [4]:
"""
Demographic data is taken from the U.S. Census Bureau's November 2016 edition of the 
Voting and Registration Supplement to the Current Population Survey:

https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-580.html

All numbers taken from these files are expressed in thousands.
"""

#Load the marginal age + gender data, taken from:
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table01.xls


census_age_gender = pd.read_excel('demographics/table01.xls')
census_age_gender.loc[census_age_gender.iloc[:, 0].isnull()]


,Table with row headers in column A and column headers in rows 4 through 6.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
3,NaN,NaN,NaN,Total Citizen Population,Reported registered,NaN,Reported not registered,NaN,No response to registration 1,NaN,Reported voted,NaN,Reported did not vote,NaN,No response to voting 2,NaN,Reported registered,Reported voted
4,NaN,NaN,NaN,NaN,Number,Percent,Number,Percent,Number,Percent,Number,Percent,Number,Percent,Number,Percent,Percent,Percent
6,NaN,18 to 24 years,29320,26913,14905,55.4,6650,24.7,5358,19.9,11560,43,10171,37.8,5182,19.3,50.8,39.4
7,NaN,25 to 34 years,43794,38283,24682,64.5,7186,18.8,6415,16.8,20332,53.1,11902,31.1,6049,15.8,56.4,46.4
8,NaN,35 to 44 years,39905,34327,23948,69.8,5280,15.4,5099,14.9,20662,60.2,8780,25.6,4885,14.2,60,51.8
9,NaN,45 to 54 years,42259,38301,28001,73.1,4682,12.2,5619,14.7,25012,65.3,7829,20.4,5460,14.3,66.3,59.2
10,NaN,55 to 64 years,41540,39242,29393,74.9,4381,11.2,5468,13.9,26657,67.9,7236,18.4,5350,13.6,70.8,64.2
11,NaN,65 to 74 years,28832,27839,21908,78.7,2502,9,3429,12.3,20219,72.6,4239,15.2,3381,12.1,76,70.1
12,NaN,75 years and over,19852,19154,14759,77.1,1941,10.1,2454,12.8,13095,68.4,3703,19.3,2356,12.3,74.3,66
13,NaN,18 years,3999,3754,1606,42.8,1173,31.2,975,26,1293,34.4,1539,41,922,24.6,40.2,32.3


In [5]:
#Load the marginal state + race + gender data, taken from:
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table04b.xls


census_race = pd.read_excel('demographics/table04b.xls')

#Convert state names from upper case to title case (only the first letter is capitalized)

census_race.iloc[15:566, 0] = census_race.iloc[15:566, 0].str.title()

#Change "District Of Columbia" to "District of Columbia" to match the survey data

census_race.iloc[103, 0] = 'District of Columbia'

#Fill in blank rows in the state column

census_race.iloc[:, 0] = census_race.iloc[:, 0].fillna(method='ffill')


#Create a separate dataframe for marginal state + gender data

census_state_gender = census_race.copy()

#Filter out race data and irrelevant columns

census_state_gender = census_state_gender.loc[census_state_gender['Unnamed: 1'].isin(['Male', 'Female'])]
census_state_gender = census_state_gender.iloc[2:, [0, 1, 3]]

#Sort so that female counts appear first

census_state_gender.columns = ['Area', 'Gender', 'Population (in thousands)']
census_state_gender.sort_values(['Area', 'Gender'], inplace=True)
census_state_gender.iloc[:, 1] = census_state_gender.iloc[:, 1].str.lower()

#Create a separate dataframe for marginal state + race data

census_state_race = census_race.copy()

#Filter out gender data and irrelevant columns

census_state_race = census_state_race.loc[census_state_race['Unnamed: 1'].isin(['Total',
                                                                                'White non-Hispanic alone',
                                                                                'Black alone',
                                                                                'Hispanic (of any race)',
                                                                                'Asian alone'])]
census_state_race = census_state_race.iloc[5: , [0, 1, 3]]
census_state_race.columns = ['Area', 'Race', 'Population (in thousands)']
census_state_race.sort_values(['Area', 'Race'],  inplace=True)


In [6]:
#Load the marginal race + gender data for each race, taken from:
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table02_3.xls
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table02_4.xls
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table02_6.xls


census_white = pd.read_excel('demographics/table02_3.xls')
census_black = pd.read_excel('demographics/table02_4.xls')
census_hispanic = pd.read_excel('demographics/table02_6.xls')


In [7]:
#Load the marginal education + gender data for each race, taken from:
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table05_1.xls


census_edu_gender = pd.read_excel('demographics/table05_1.xls')


In [8]:
#Load the state data, taken from:
#https://www2.census.gov/programs-surveys/cps/tables/p20/580/table04a.xls


census_state = pd.read_excel('demographics/table04a.xls')

#Convert state names from upper case to title case (only the first letter is capitalized)

census_state.iloc[5:56, 0] = census_state.iloc[5:56, 0].str.title()

#Change "District Of Columbia" to "District of Columbia" to match the survey data

census_state.iloc[13, 0] = 'District of Columbia'


In [9]:
#Calculate the frequencies for gender, race, and age

gender_freq = pd.Series(data=[census_age_gender.iloc[149, 3],
                              census_age_gender.iloc[77, 3]],
                        index=['female', 'male'])

race_freq = pd.Series(data=[census_race.iloc[8, 3],
                            census_race.iloc[4, 3] - census_race.iloc[[8, 9, 11], 3].sum(),
                            census_race.iloc[11, 3],
                            census_race.iloc[9, 3]],
                      index=['white', 'other', 'hispanic', 'black'])

age_freq = pd.Series(data=[census_age_gender.iloc[6, 3],
                           census_age_gender.iloc[7, 3],
                           census_age_gender.iloc[8, 3],
                           census_age_gender.iloc[9, 3],
                           census_age_gender.iloc[10:13, 3].sum()],
                     index=['18 - 24', '25 - 34', '35 - 44', '45 - 54', '> 54'])

edu_freq = pd.Series(data=[census_edu_gender.iloc[6:10, 3].sum(),
                           census_edu_gender.iloc[10:12, 3].sum()],
                     index=['bachelors', 'no bachelors'])

state_freq = census_state.iloc[5:56, 2]
state_freq.index = census_state.iloc[5:56, 0]


In [10]:
#Group the data by race and gender

race_gender_freq = common_data.groupby(['Race', 'Gender']).size()

#Replace each marginal frequency cell with data taken from the Census Bureau data

race_gender_freq['white'] = census_white.iloc[[11, 17], 3]
race_gender_freq['black'] = census_black.iloc[[11, 17], 3]
race_gender_freq['hispanic'] = census_hispanic.iloc[[11, 17], 3]

#Add the marginal frequencies from the white, black, and hispanic populations together

non_other_gender = census_hispanic.iloc[[11, 17], 3] + census_black.iloc[[11, 17], 3] + census_white.iloc[[11, 17], 3]
non_other_gender.index = ['female', 'male']

#Find the marginal frequencies for race "other" by subtracting the "non-other" marginal frequencies from the gender totals

race_gender_freq['other'] = gender_freq - non_other_gender


In [11]:
#Group the sample data by age and gender

age_gender_freq = common_data.groupby(['Age', 'Gender']).size()

#Replace each marginal frequency cell with data taken from the Census Bureau data

age_gender_freq.loc['18 - 24'] = [census_age_gender.iloc[150, 3], census_age_gender.iloc[78, 3]]
age_gender_freq.loc['25 - 34'] = [census_age_gender.iloc[151, 3], census_age_gender.iloc[79, 3]]
age_gender_freq.loc['35 - 44'] = [census_age_gender.iloc[152, 3], census_age_gender.iloc[80, 3]]
age_gender_freq.loc['45 - 54'] = [census_age_gender.iloc[153, 3], census_age_gender.iloc[81, 3]]

#Combine the marginal frequencies for 55-64, 65-74, and >75 to form the >54 marginal frequencies

age_gender_freq.loc['> 54'] = [census_age_gender.iloc[154:157, 3].sum(),
                    census_age_gender.iloc[82:85, 3].sum()]


In [12]:
#Group the sample data by education and gender

edu_gender_freq = common_data.groupby(['Education', 'Gender']).size()

#Combine the marginal frequencies for "Less than 9th grade" to "Some college or associate degree" to form the "no bachelors" marginal frequencies

edu_gender_freq.loc['bachelors'] = [census_edu_gender.iloc[24:26, 3].sum(),
                                    census_edu_gender.iloc[17:19, 3].sum()]

#Combine the marginal frequencies for "Bachelor's degree" and "Advanced degree" to form the "bachelors" marginal frequencies

edu_gender_freq.loc['no bachelors'] = [census_edu_gender.iloc[20:24, 3].sum(),
                                       census_edu_gender.iloc[13:17, 3].sum()]


In [13]:
#Group the sample data by state and gender

gender_state_freq = common_data.groupby(['Area', 'Gender']).size()

#Replace the counts in the aggregated table with the Census counts

census_state_gender = census_state_gender.iloc[:, 2]
census_state_gender.index = gender_state_freq.index
gender_state_freq = census_state_gender.copy()


In [14]:
#Calculate the population count for the race "other" for each state

census_state_race.loc[census_state_race['Race'] == 'Asian alone', 'Race'] = 'other'
census_state_race.reset_index(drop=True, inplace=True)

for row in census_state_race.iterrows():
    if row[0]%5 == 0:
        non_other = census_state_race.iloc[[row[0] + 1, row[0] + 2, row[0] + 4], 2].sum()
        total = census_state_race.iloc[row[0] + 3, 2]
        row[1]['Population (in thousands)'] = total - non_other
        
#Drop the population total rows for each state

census_state_race = census_state_race.loc[census_state_race['Race'] != 'Total']

#Rename the race column values to match the survey data

census_state_race.loc[census_state_race['Race'] == 'Black alone', 'Race'] = 'black'
census_state_race.loc[census_state_race['Race'] == 'Hispanic (of any race)', 'Race'] = 'hispanic'
census_state_race.loc[census_state_race['Race'] == 'White non-Hispanic alone', 'Race'] = 'white'

census_state_race.sort_values(['Area', 'Race'], inplace=True)


In [ ]:
#Enumerate the each possible combination of state and race

marginals = [marginal for marginal in product(census_state_race['Area'].unique(), census_state_race['Race'].unique())]

#Index the dataframe by state and race

census_state_race = census_state_race.iloc[:, 2]
index = pd.MultiIndex.from_tuples(marginals, names=['Area', 'Race'])
race_state_freq = pd.Series(census_state_race.values, index=index)


In [ ]:
#Create a list of the marginal/joint frequencies obtained from the census data

aggregates = [gender_freq,
              race_freq,
              age_freq,
              state_freq,
              age_gender_freq,
              race_gender_freq,
              edu_gender_freq,
              gender_state_freq,
#              race_state_freq
              ]

#Create a list of column names that correspond to the frequency tables in the previous list

dimensions = [['Gender'],
              ['Race'],
              ['Age'],
              ['Area'],
              ['Age', 'Gender'],
              ['Race', 'Gender'],
              ['Education', 'Gender'],
              ['Area', 'Gender'],
#              ['Area', 'Race']
             ]

#Perform raking on the demographic data up to the specified maximum number of iterations

IPF = ipfn.ipfn(freqs, aggregates, dimensions, max_iteration=5000)
df = IPF.iteration()

df.columns = ['Area', 'Race', 'Gender', 'Education', 'Age', 'Population (in thousands)']
df.to_csv('python_raking_race_gender_age_edu_state.csv', 
          columns=['Area', 'Race', 'Gender', 'Education', 'Age', 'Population (in thousands)'],
          index=False)
print(df)
['Area', 'Gender', 'Race', 'Age', 'Education', 'total']

/home/handsomemachine/anaconda3/envs/MSR/lib/python3.6/site-packages/pandas/core/indexing.py:1325: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/handsomemachine/anaconda3/envs/MSR/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/home/handsomemachine/anaconda3/envs/MSR/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: PerformanceWarning: indexing past lexsort depth may impact performance.
  if self.run_code(code, result):


In [ ]:
crosstab = pd.crosstab(index=[df['Age'], df['Gender']], columns=df['Race'],
                       values=df['Population (in thousands)'], aggfunc='sum')
crosstab.to_csv('python_raking_crosstab_state.csv')
